In [52]:
# Standard library imports
from collections import defaultdict
from itertools import permutations
import copy

# Third-party imports
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pygraphviz
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import rpy2py
from scipy import stats
from scipy.stats import t
import statsmodels.api as sm

import papermill as pm
import nbformat
import pandas as pd
from papermill.exceptions import PapermillExecutionError
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import json
import numpy as np
import os

In [53]:
def extract_values_from_notebook(output_notebook_path):
    nb = nbformat.read(output_notebook_path, as_version=4)
    for cell in nb.cells:
        if cell.cell_type == 'code':
            for output in cell.get('outputs', []):
                if output.output_type == 'stream' and 'name' in output and output['name'] == 'stdout':
                    if "algorithm_result" in output['text']:
                        # Assuming the output is JSON-formatted string following "algorithm_result"
                        result_str = output['text'].split("algorithm_result")[1].strip()
                        result_dict = json.loads(result_str)
                        return result_dict
    return None

In [54]:
# Enable the conversion between Pandas DataFrames and R data.frames


k_list =[15, 20, 25, 30]

results = []
nb_path = "/Users/takeshine/Desktop/new_layer/algorithm_new_layer.ipynb"


for node in k_list:
    ro.r.source("generate_graph.R")
    k = node
    # Access the R function from the global environment
    generateGraphWithHiddenNodes = ro.globalenv['generateGraphWithHiddenNodes']

    # Call the R function with k as argument, and adjust to handle the function's updated return type
    result = generateGraphWithHiddenNodes(k)

    # Extract the graph and hidden nodes from the result
    g_directed = result.rx2('Graph')
    hidden_nodes = result.rx2('HiddenNodes')

    for i in range(20):
        ro.r.source("/Users/takeshine/Desktop/new_layer/process_graph.R")
        processGraph = ro.globalenv['processGraph']

        processed_result = processGraph(g_directed, k, hidden_nodes)

        returned_list = result[0]

        # x_data = rpy2py(returned_list[0]) 
        x_data = rpy2py(processed_result.rx2('x_data'))


        if not isinstance(x_data, pd.DataFrame):
            x_data = pd.DataFrame(x_data)

        x_data_path = "/Users/takeshine/Desktop/new_layer/data_set/x_data.csv"
        x_data.to_csv(x_data_path, index=False)
       

        # original_edge_data = rpy2py(returned_list[1]) 
        original_edge_data = rpy2py(processed_result.rx2('original_edge_data'))
        pandas2ri.activate()
        df_edge = pandas2ri.rpy2py(original_edge_data)
        df_edge = pd.DataFrame(df_edge)
        df_edge.columns = ['source', 'target']
        df_edge['source'] = df_edge['source'].astype(str)
        df_edge['target'] = df_edge['target'].astype(str)


        # if not isinstance(original_edge_data, pd.DataFrame):
           # original_edge_data = pd.DataFrame(original_edge_data, columns=['source', 'target'])

        original_edge_data_path = "/Users/takeshine/Desktop/new_layer/data_set/original_edge_data.csv" 
        df_edge.to_csv(original_edge_data_path,index=False)


        sample_sizes = [100000, 10000, 1000, 100, 10]

        for sample_size in sample_sizes:
            output_path = f'/Users/takeshine/Desktop/new_layer/output/prefix_{sample_size}_index_{i}_node_{node}.ipynb'

            try:
                pm.execute_notebook(
                    nb_path,
                    output_path,
                    parameters={
                        'original_edge_data_path': original_edge_data_path,
                        'x_data_path': x_data_path,
                        'sample_size': sample_size  # Pass the sample size

                    }
                )
                # extract results 
                results_dict = extract_values_from_notebook(output_path)

                if results_dict is not None:
                    results.append({
                        'tree_size': node,
                        'lambda_index': i+1,
                        'sample_size': sample_size,
                         **results_dict
                    })
            except PapermillExecutionError as e:
                print(f"Error executing the notebook with lambda_index={i+1}, node_{node},sample_size={sample_size}: {e}")

Exception ignored from cffi callback <function _processevents at 0x140c20180>:
Traceback (most recent call last):
  File "/Users/takeshine/Downloads/anaconda3/envs/Thesis/lib/python3.11/site-packages/rpy2/rinterface_lib/callbacks.py", line 275, in _processevents
    @ffi_proxy.callback(ffi_proxy._processevents_def,
    
KeyboardInterrupt: 
Exception ignored from cffi callback <function _processevents at 0x140c20180>:
Traceback (most recent call last):
  File "/Users/takeshine/Downloads/anaconda3/envs/Thesis/lib/python3.11/site-packages/rpy2/rinterface_lib/callbacks.py", line 275, in _processevents
    @ffi_proxy.callback(ffi_proxy._processevents_def,
    
KeyboardInterrupt: 


In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df['percentage_hidden_nodes_recovered'] = results_df['percentage_hidden_nodes_recovered']*100
results_df['Normalized SHD'] = results_df['shd'] / (results_df['number_of_edges_esitimation'] + results_df['number_of_edges_true'])

In [ ]:
df_result_path = "/Users/takeshine/Desktop/new_layer/result_df.csv"
results_df.to_csv(df_result_path, index=False)

In [ ]:
results_df

,tree_size,lambda_index,sample_size,shd,precision,recall,percentage_hidden_nodes_recovered,number_of_edges_esitimation,number_of_edges_true,Normalized SHD
0,15,1,100000,15,0.363636,0.333333,133.333333,11,12,0.652174
1,15,1,10000,6,0.875000,0.583333,0.000000,8,12,0.300000
2,15,1,1000,13,0.454545,0.416667,66.666667,11,12,0.565217
3,15,1,100,11,0.555556,0.416667,33.333333,9,12,0.523810
4,15,1,10,12,0.500000,0.083333,0.000000,2,12,0.857143
...,...,...,...,...,...,...,...,...,...,...
395,30,20,100000,18,0.680000,0.629630,120.000000,25,27,0.346154
396,30,20,10000,30,0.444444,0.444444,180.000000,27,27,0.555556
397,30,20,1000,20,0.629630,0.629630,140.000000,27,27,0.370370
398,30,20,100,31,0.416667,0.370370,180.000000,24,27,0.607843


Avereage Results for all trees